In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./item.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_item_number = df['Item_Number'].tolist()
list_category_id = df['Category_Id'].tolist()
list_market_id = df['Market_Id'].tolist()
list_vehicle_page = df['Vehicle_Page'].tolist()

print('总数量：' + str(len(list_item_number)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

dict_vehicle = {}

# = = = = = = = = = = = = = = =

list_job = []
for item_number, category_id, market_id, vehicle_page in zip(list_item_number, list_category_id, list_market_id, list_vehicle_page):
    if vehicle_page == '':
        df_temp = pd.DataFrame([{'status': 'ok',
                                 'Item_Number': item_number,
                                 'Vehicle': ''}])
        
        df = pd.concat([df, df_temp], ignore_index=True).fillna('')
    else:
        dict_vehicle[item_number] = {}
        dict_vehicle[item_number]['vehicle_page'] = int(vehicle_page)
        for i in range(int(vehicle_page)):
            list_job.append((item_number, category_id, market_id, i))
            
print('总数量：' + str(len(list_job)))
print()

# = = = = = = = = = = = = = = =

import requests

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for job in list_job:
    work.put_nowait(job)

def crawler():
    global df
    
    while not work.empty():
        item_number, category_id, market_id, vehicle_page = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                data = '{"scopedContext":{"catalogDetails":{"itemId":"' + item_number + '","categoryId":"' + category_id + '","marketplaceId":"' + market_id + '"}}}'

                ua = UA.get_User_Agent_Requests()
                ua['Content-Type'] = 'application/json'

                resp = requests.post('https://www.ebay.com/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(vehicle_page*20) + '&module=COMPATIBILITY_TABLE&_ul=US&_stpos=91710&orig_cvip=true',
                                     data=data,
                                     headers=ua,
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)

                # = = = = = = = = = = = = = = =

                list_header = resp['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']

                # = = = = = = = = = = = = = = =

                for k in range(len(list_header)):
                    if 'Make' in list_header[k]['textSpans'][0]['text']:
                        col_make = k
                    elif 'Model' in list_header[k]['textSpans'][0]['text']:
                        col_model = k
                    elif 'Year' in list_header[k]['textSpans'][0]['text']:
                        col_year = k

                # = = = = = = = = = = = = = = =

                list_row = resp['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']

                # = = = = = = = = = = = = = = =

                list_vehicle = []
                for row in list_row:
                    make = row['cells'][col_make]['textSpans'][0]['text'].strip()
                    model = row['cells'][col_model]['textSpans'][0]['text'].strip()
                    year = int(row['cells'][col_year]['textSpans'][0]['text'].strip())

                    list_vehicle.append({'make': make, 'model': model, 'year': year})
                    
                dict_vehicle[item_number][str(vehicle_page)] = list_vehicle
                
                status = 'ok'
                
                # = = = = = = = = = = = = = = =
                
                finish = False
                if dict_vehicle[item_number]['vehicle_page']+1 == len(dict_vehicle[item_number]):
                    new_list_vehicle = []
                    new_dict_vehicle = {}
                    for i in range(dict_vehicle[item_number]['vehicle_page']):
                        list_vehicle += dict_vehicle[item_number][str(i)]

                    for vehicle in list_vehicle:
                        if vehicle['make'] not in new_dict_vehicle:
                            new_dict_vehicle[vehicle['make']] = {vehicle['model']: [vehicle['year']]}
                        else:
                            if vehicle['model'] not in new_dict_vehicle[vehicle['make']]:
                                new_dict_vehicle[vehicle['make']][vehicle['model']] = [vehicle['year']]
                            else:
                                new_dict_vehicle[vehicle['make']][vehicle['model']].append(vehicle['year'])
                
                    # = = = = = = = = = = = = = = =
                    
                    new_list_vehicle = []
                    for make in new_dict_vehicle:
                        for model in new_dict_vehicle[make]:
                            year_max = max(new_dict_vehicle[make][model])
                            year_min = min(new_dict_vehicle[make][model])
                            if year_max == year_min:
                                new_list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                            else:
                                new_list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                    vehicle = '\n'.join(new_list_vehicle)
                
                    # = = = = = = = = = = = = = = =

                    df_temp = pd.DataFrame([{'status': 'ok',
                                             'Item_Number': item_number,
                                             'Vehicle': vehicle}])
                    
                    df = pd.concat([df, df_temp], ignore_index=True).fillna('')

                    finish = True
                
                break
                
            except:
                time.sleep(1)
                continue

        print(item_number + ' ' + str(vehicle_page) +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))
        if finish:
            print(item_number + ' 已全部获取！！！')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(18):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./vehicle_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：15412

总数量：232806

231999231613 5  <->  [ok] - 剩余数量：232790 - 当前时间：10:43:05
231999231613 3  <->  [ok] - 剩余数量：232789 - 当前时间：10:43:05
231999231613 8  <->  [ok] - 剩余数量：232788 - 当前时间：10:43:05
231999231613 13  <->  [ok] - 剩余数量：232787 - 当前时间：10:43:05
231999231613 12  <->  [ok] - 剩余数量：232786 - 当前时间：10:43:05
231999231613 0  <->  [ok] - 剩余数量：232785 - 当前时间：10:43:05
231999231613 10  <->  [ok] - 剩余数量：232784 - 当前时间：10:43:05
231999231613 15  <->  [ok] - 剩余数量：232783 - 当前时间：10:43:05
231999231613 11  <->  [ok] - 剩余数量：232782 - 当前时间：10:43:05
231999231613 7  <->  [ok] - 剩余数量：232781 - 当前时间：10:43:05
231999231613 4  <->  [ok] - 剩余数量：232780 - 当前时间：10:43:05
231999231613 2  <->  [ok] - 剩余数量：232779 - 当前时间：10:43:05
231999231613 6  <->  [ok] - 剩余数量：232778 - 当前时间：10:43:05
231999231613 9  <->  [ok] - 剩余数量：232777 - 当前时间：10:43:05
231999231613 14  <->  [ok] - 剩余数量：232776 - 当前时间：10:43:06
231999231613 1  <->  [ok] - 剩余数量：232775 - 当前时间：10:43:06
231999231613 16  <->  [ok] - 剩余数量：232774 - 当前时间：10:43:06
231999231613 20  <

231999231613 146  <->  [ok] - 剩余数量：232647 - 当前时间：10:43:20
232789513043 5  <->  [ok] - 剩余数量：232646 - 当前时间：10:43:21
232789513043 6  <->  [ok] - 剩余数量：232645 - 当前时间：10:43:21
231999231613 147  <->  [ok] - 剩余数量：232644 - 当前时间：10:43:21
231999231613 111  <->  [ok] - 剩余数量：232643 - 当前时间：10:43:21
232789513043 8  <->  [ok] - 剩余数量：232642 - 当前时间：10:43:21
232789513043 3  <->  [ok] - 剩余数量：232641 - 当前时间：10:43:21
232789513043 1  <->  [ok] - 剩余数量：232640 - 当前时间：10:43:21
232789513043 10  <->  [ok] - 剩余数量：232639 - 当前时间：10:43:22
231999231613 138  <->  [ok] - 剩余数量：232638 - 当前时间：10:43:22
232789513043 11  <->  [ok] - 剩余数量：232637 - 当前时间：10:43:22
232789513043 15  <->  [ok] - 剩余数量：232636 - 当前时间：10:43:22
232789513043 7  <->  [ok] - 剩余数量：232635 - 当前时间：10:43:23
232789513043 16  <->  [ok] - 剩余数量：232634 - 当前时间：10:43:23
232789513043 18  <->  [ok] - 剩余数量：232633 - 当前时间：10:43:23
232789513043 17  <->  [ok] - 剩余数量：232632 - 当前时间：10:43:23
232789513043 14  <->  [ok] - 剩余数量：232631 - 当前时间：10:43:23
232789513043 13  <->  [ok] - 剩余数量

232789513043 142  <->  [ok] - 剩余数量：232503 - 当前时间：10:43:45
232789513043 149  <->  [ok] - 剩余数量：232502 - 当前时间：10:43:45
232789513043 148  <->  [ok] - 剩余数量：232501 - 当前时间：10:43:46
233046655699 1  <->  [ok] - 剩余数量：232500 - 当前时间：10:43:46
233046655699 0  <->  [ok] - 剩余数量：232499 - 当前时间：10:43:46
232789513043 76  <->  [ok] - 剩余数量：232498 - 当前时间：10:43:46
233046655699 2  <->  [ok] - 剩余数量：232497 - 当前时间：10:43:46
233046655699 3  <->  [ok] - 剩余数量：232496 - 当前时间：10:43:46
233046655699 5  <->  [ok] - 剩余数量：232495 - 当前时间：10:43:47
232789513043 121  <->  [ok] - 剩余数量：232494 - 当前时间：10:43:47
233046655699 8  <->  [ok] - 剩余数量：232493 - 当前时间：10:43:47
231999231613 129  <->  [ok] - 剩余数量：232492 - 当前时间：10:43:48
233046655699 7  <->  [ok] - 剩余数量：232491 - 当前时间：10:43:48
233046655699 10  <->  [ok] - 剩余数量：232490 - 当前时间：10:43:48
233046655699 11  <->  [ok] - 剩余数量：232489 - 当前时间：10:43:48
233046655699 9  <->  [ok] - 剩余数量：232488 - 当前时间：10:43:48
233046655699 6  <->  [ok] - 剩余数量：232487 - 当前时间：10:43:48
233046655699 4  <->  [ok] - 剩余数量：23

233046655699 95  <->  [ok] - 剩余数量：232360 - 当前时间：10:44:12
233046655699 144  <->  [ok] - 剩余数量：232359 - 当前时间：10:44:12
233046655699 145  <->  [ok] - 剩余数量：232358 - 当前时间：10:44:12
233046655699 141  <->  [ok] - 剩余数量：232357 - 当前时间：10:44:12
233046655699 142  <->  [ok] - 剩余数量：232356 - 当前时间：10:44:13
233046655699 140  <->  [ok] - 剩余数量：232355 - 当前时间：10:44:13
233046655699 143  <->  [ok] - 剩余数量：232354 - 当前时间：10:44:13
233046655699 148  <->  [ok] - 剩余数量：232353 - 当前时间：10:44:13
233046655699 146  <->  [ok] - 剩余数量：232352 - 当前时间：10:44:14
233046655699 147  <->  [ok] - 剩余数量：232351 - 当前时间：10:44:14
233061703639 2  <->  [ok] - 剩余数量：232350 - 当前时间：10:44:14
233061703639 3  <->  [ok] - 剩余数量：232349 - 当前时间：10:44:14
233061703639 0  <->  [ok] - 剩余数量：232348 - 当前时间：10:44:14
233061703639 4  <->  [ok] - 剩余数量：232347 - 当前时间：10:44:15
233061703639 5  <->  [ok] - 剩余数量：232346 - 当前时间：10:44:15
233061703639 1  <->  [ok] - 剩余数量：232345 - 当前时间：10:44:15
233046655699 149  <->  [ok] - 剩余数量：232344 - 当前时间：10:44:15
233061703639 6  <->  [ok] -

233061703639 137  <->  [ok] - 剩余数量：232215 - 当前时间：10:44:45
233061703639 136  <->  [ok] - 剩余数量：232214 - 当前时间：10:44:46
233061703639 138  <->  [ok] - 剩余数量：232213 - 当前时间：10:44:46
233061703639 139  <->  [ok] - 剩余数量：232212 - 当前时间：10:44:46
233061703639 140  <->  [ok] - 剩余数量：232211 - 当前时间：10:44:46
233061703639 141  <->  [ok] - 剩余数量：232210 - 当前时间：10:44:47
233061703639 142  <->  [ok] - 剩余数量：232209 - 当前时间：10:44:47
233061703639 145  <->  [ok] - 剩余数量：232208 - 当前时间：10:44:47
233061703639 143  <->  [ok] - 剩余数量：232207 - 当前时间：10:44:48
233061703639 144  <->  [ok] - 剩余数量：232206 - 当前时间：10:44:48
233061703639 146  <->  [ok] - 剩余数量：232205 - 当前时间：10:44:48
233061703639 147  <->  [ok] - 剩余数量：232204 - 当前时间：10:44:48
233061703639 148  <->  [ok] - 剩余数量：232203 - 当前时间：10:44:48
233061712634 1  <->  [ok] - 剩余数量：232202 - 当前时间：10:44:49
233061703639 149  <->  [ok] - 剩余数量：232201 - 当前时间：10:44:49
233061712634 3  <->  [ok] - 剩余数量：232200 - 当前时间：10:44:49
233061712634 2  <->  [ok] - 剩余数量：232199 - 当前时间：10:44:49
233061712634 4  <-> 

...

235338270217 3  <->  [ok] - 剩余数量：191934 - 当前时间：14:06:06
235331543172 1  <->  [ok] - 剩余数量：191933 - 当前时间：14:06:07
235331543172 已全部获取！！！
235338270217 6  <->  [ok] - 剩余数量：191932 - 当前时间：14:06:07
235338270217 2  <->  [ok] - 剩余数量：191931 - 当前时间：14:06:07
235338270220 0  <->  [ok] - 剩余数量：191930 - 当前时间：14:06:07
235338270220 1  <->  [ok] - 剩余数量：191929 - 当前时间：14:06:07
235338268561 0  <->  [ok] - 剩余数量：191928 - 当前时间：14:06:07
235338268561 已全部获取！！！
235338270217 5  <->  [ok] - 剩余数量：191927 - 当前时间：14:06:08
235338270220 2  <->  [ok] - 剩余数量：191926 - 当前时间：14:06:08
235338270220 4  <->  [ok] - 剩余数量：191925 - 当前时间：14:06:08
235338270220 6  <->  [ok] - 剩余数量：191924 - 当前时间：14:06:08
235338271254 0  <->  [ok] - 剩余数量：191923 - 当前时间：14:06:08
235338271254 已全部获取！！！
235338271525 0  <->  [ok] - 剩余数量：191922 - 当前时间：14:06:08
235338271525 1  <->  [ok] - 剩余数量：191921 - 当前时间：14:06:09
235338270222 0  <->  [ok] - 剩余数量：191920 - 当前时间：14:06:09
235338270222 已全部获取！！！
235338271525 2  <->  [ok] - 剩余数量：191919 - 当前时间：14:06:10
235338270226 0  

235338274209 1  <->  [ok] - 剩余数量：191793 - 当前时间：14:06:42
235338274209 0  <->  [ok] - 剩余数量：191792 - 当前时间：14:06:42
235338274209 已全部获取！！！
235338274292 0  <->  [ok] - 剩余数量：191791 - 当前时间：14:06:42
235338274292 已全部获取！！！
235338274297 0  <->  [ok] - 剩余数量：191790 - 当前时间：14:06:42
235338274297 已全部获取！！！
235338275850 0  <->  [ok] - 剩余数量：191789 - 当前时间：14:06:42
235338275850 1  <->  [ok] - 剩余数量：191788 - 当前时间：14:06:42
235338275850 2  <->  [ok] - 剩余数量：191787 - 当前时间：14:06:43
235338275850 3  <->  [ok] - 剩余数量：191786 - 当前时间：14:06:43
235338275850 5  <->  [ok] - 剩余数量：191785 - 当前时间：14:06:43
235338275850 4  <->  [ok] - 剩余数量：191784 - 当前时间：14:06:43
235338275850 6  <->  [ok] - 剩余数量：191783 - 当前时间：14:06:43
235338275850 7  <->  [ok] - 剩余数量：191782 - 当前时间：14:06:44
235338273489 8  <->  [ok] - 剩余数量：191781 - 当前时间：14:06:44
235338275906 2  <->  [ok] - 剩余数量：191780 - 当前时间：14:06:44
235338275850 8  <->  [ok] - 剩余数量：191779 - 当前时间：14:06:44
235338275850 已全部获取！！！
235338275906 0  <->  [ok] - 剩余数量：191778 - 当前时间：14:06:45
235338273489 6  

235348759489 7  <->  [ok] - 剩余数量：191653 - 当前时间：14:07:09
235348759489 12  <->  [ok] - 剩余数量：191652 - 当前时间：14:07:09
235348759747 0  <->  [ok] - 剩余数量：191651 - 当前时间：14:07:10
235348759747 1  <->  [ok] - 剩余数量：191650 - 当前时间：14:07:10
235348759747 2  <->  [ok] - 剩余数量：191649 - 当前时间：14:07:10
235348759747 3  <->  [ok] - 剩余数量：191648 - 当前时间：14:07:10
235348759747 已全部获取！！！
235348760118 2  <->  [ok] - 剩余数量：191647 - 当前时间：14:07:10
235348760118 1  <->  [ok] - 剩余数量：191646 - 当前时间：14:07:11
235348760118 5  <->  [ok] - 剩余数量：191645 - 当前时间：14:07:11
235348760118 4  <->  [ok] - 剩余数量：191644 - 当前时间：14:07:11
235348760118 7  <->  [ok] - 剩余数量：191643 - 当前时间：14:07:12
235348760118 8  <->  [ok] - 剩余数量：191642 - 当前时间：14:07:12
235348760118 10  <->  [ok] - 剩余数量：191641 - 当前时间：14:07:12
235338273489 44  <->  [ok] - 剩余数量：191640 - 当前时间：14:07:12
235338273489 已全部获取！！！
235348760118 9  <->  [ok] - 剩余数量：191639 - 当前时间：14:07:12
235348760118 13  <->  [ok] - 剩余数量：191638 - 当前时间：14:07:13
235348760118 14  <->  [ok] - 剩余数量：191637 - 当前时间：14:07:13

235348770501 25  <->  [ok] - 剩余数量：191509 - 当前时间：14:07:45
235348770501 20  <->  [ok] - 剩余数量：191508 - 当前时间：14:07:46
235348770501 16  <->  [ok] - 剩余数量：191507 - 当前时间：14:07:46
235348770501 29  <->  [ok] - 剩余数量：191506 - 当前时间：14:07:46
235348770501 31  <->  [ok] - 剩余数量：191505 - 当前时间：14:07:47
235348770501 30  <->  [ok] - 剩余数量：191504 - 当前时间：14:07:47
235348770501 26  <->  [ok] - 剩余数量：191503 - 当前时间：14:07:47
235348770501 33  <->  [ok] - 剩余数量：191502 - 当前时间：14:07:47
235348770501 27  <->  [ok] - 剩余数量：191501 - 当前时间：14:07:47
235348770501 34  <->  [ok] - 剩余数量：191500 - 当前时间：14:07:48
235348770501 35  <->  [ok] - 剩余数量：191499 - 当前时间：14:07:48
235348770501 36  <->  [ok] - 剩余数量：191498 - 当前时间：14:07:48
235348770501 40  <->  [ok] - 剩余数量：191497 - 当前时间：14:07:49
235348770501 39  <->  [ok] - 剩余数量：191496 - 当前时间：14:07:49
235348792900 0  <->  [ok] - 剩余数量：191495 - 当前时间：14:07:49
235348792900 1  <->  [ok] - 剩余数量：191494 - 当前时间：14:07:50
235348792900 已全部获取！！！
235348792951 0  <->  [ok] - 剩余数量：191493 - 当前时间：14:07:50
235348792951

235348795053 50  <->  [ok] - 剩余数量：191369 - 当前时间：14:08:36
235348795053 51  <->  [ok] - 剩余数量：191368 - 当前时间：14:08:37
235348795053 52  <->  [ok] - 剩余数量：191367 - 当前时间：14:08:37
235348795053 55  <->  [ok] - 剩余数量：191366 - 当前时间：14:08:37
235348795053 54  <->  [ok] - 剩余数量：191365 - 当前时间：14:08:37
235348795053 58  <->  [ok] - 剩余数量：191364 - 当前时间：14:08:38
235348795053 57  <->  [ok] - 剩余数量：191363 - 当前时间：14:08:38
235348795053 59  <->  [ok] - 剩余数量：191362 - 当前时间：14:08:38
235348795053 56  <->  [ok] - 剩余数量：191361 - 当前时间：14:08:39
235348795053 61  <->  [ok] - 剩余数量：191360 - 当前时间：14:08:39
235348795053 62  <->  [ok] - 剩余数量：191359 - 当前时间：14:08:39
235348795053 64  <->  [ok] - 剩余数量：191358 - 当前时间：14:08:40
235348795053 63  <->  [ok] - 剩余数量：191357 - 当前时间：14:08:40
235348795053 65  <->  [ok] - 剩余数量：191356 - 当前时间：14:08:40
235348795053 66  <->  [ok] - 剩余数量：191355 - 当前时间：14:08:40
235348795053 67  <->  [ok] - 剩余数量：191354 - 当前时间：14:08:40
235348795053 69  <->  [ok] - 剩余数量：191353 - 当前时间：14:08:41
235348795053 68  <->  [ok] - 剩余